# Power BI Measure Extractor 
## Extract all used measures, their names, and data sources within Power BI Reports
This Script extracts the Measures in the following format

| Berichtsname | Berichtseite | Visualisierung | Name Measure | Datenquelle |
| ------------ | ------------ | -------------- | ------------ | ----------- |
| Name Bericht | Seite 1 | Name der Grafik | Name Measure | #_Measure.Beispiel |

Before you can extract the measures it is required to put only the needed *.pbix files* into one folder together with the Notebook, as every file .pbix within the folder will be considered for measure extraction.

In [31]:
import json
import pandas as pd
import os
import zipfile
from datetime import datetime

In [32]:
# Function to extract and reformat the information
def extract_info(json_data, berichtsname):
    extracted_info = []
    
    for section in json_data.get('sections', []):
        section_id = section.get('id')
        display_name = section.get('displayName')
        
        for container in section.get('visualContainers', []):
            config_str = container.get('config')
            if config_str:
                try:
                    config_data = json.loads(config_str)
                    single_visual = config_data.get('singleVisual', {})
                    
                    # Check if the visualType is 'slicer' or 'textbox'
                    visual_type = single_visual.get('visualType')
                    if visual_type in ['slicer', 'textbox']:
                        continue  # Skip this iteration if it's a slicer or textbox

                    # Extract title text / Visualisierung (Name)
                    title_objects = single_visual.get('vcObjects', {}).get('title', [])
                    title_text = ''
                    for title_obj in title_objects:
                        title_text = title_obj.get('properties', {}).get('text', {}).get('expr', {}).get('Literal', {}).get('Value', '').strip("'")
                        if title_text:
                            break  # Exit loop once we find the first title text

                    # Extract select items / Datenquelle
                    column_properties = single_visual.get('columnProperties', {})
                    measures = []
                    # Add measures to extracted info
                    for item in single_visual.get('prototypeQuery', {}).get('Select', []):
                        name = item.get('Name')
                        property_name = item.get('Column', {}).get('Property')
                        display_name_property = column_properties.get(name, {}).get('displayName', property_name)
                        
                        # Collect measures
                        if display_name_property:
                            measure_name = display_name_property
                        elif name:
                            measure_name = name.split('.')[-1]
                        
                        # Add each measure with its corresponding Datenquelle
                        extracted_info.append({
                            'Berichtsname': berichtsname,
                            'section_id': section_id,
                            'Berichtseite': display_name,
                            'Visualisierung': title_text,
                            'Name Measure': measure_name,
                            'Datenquelle': name,  # This now correctly represents the source for each measure
                        })

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                except Exception as e:
                    print(f"An error occurred: {e}")

    return extracted_info

# Create a DataFrame
df=pd.DataFrame()

In [ ]:
directory = os.getcwd()

infos = []

# Loop through each file in the specified directory
for filename in os.listdir(directory):
    if filename.endswith('.pbix'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
           fileInfo = zip_ref.getinfo(name="Report/Layout")
           layoutJSONBytes = zip_ref.read(name=fileInfo)
           layoutJSONString = layoutJSONBytes.decode("utf-16-le")
           extracted_info = extract_info(json.loads(layoutJSONString), filename.replace(".pbix", ""))
           infos += extracted_info

df=pd.DataFrame(infos)
df = df[['Berichtsname', 'Berichtseite', 'Visualisierung', 'Name Measure', 'Datenquelle']]

#display(df)

# Create a timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the CSV file name with the timestamp
csv_file_name = f"Extract_PBI_Measures_{timestamp}.csv"

df.to_csv(csv_file_name, index=False, encoding="utf-16")

print(f"DataFrame saved as {csv_file_name}")